## Final pipeline

In [188]:
# Import main packages
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import os
from skimage.color import rgb2hsv
from skimage.morphology import closing, opening, disk
from skimage import io
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pdob

# 1. Define functions

In [189]:
def downsample_images(file_path, save_path, size=(1500, 1000)):
    im = Image.open(file_path)
    downsampled_image = im.resize(size)  # Specify desired width and height

    # Create the save directory if it does not exist
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    
    # Construct the save path for the downsampled image
    filename = os.path.basename(file_path)
    downsampled_image_save_path = os.path.join(save_path, filename)
    
    # Save the downsampled image
    downsampled_image.save(downsampled_image_save_path)

def downsample_images_in_directory(input_directory, output_directory, size=(1500, 1000)):
    # List all JPG images in the directory
    images = glob.glob(os.path.join(input_directory, '*.JPG'))
    
    for image_path in images:
        downsample_images(image_path, output_directory, size)
        print(f"Processed and saved: {os.path.basename(image_path)}")

# Example usage
input_directory = "C:/Users/rapha/OneDrive/Documents/2. EPFL/Master/MA2/Image Analysis & Pattern Recognition/iapr24-coin-counter/train/3. hand"
output_directory = "C:/Users/rapha/OneDrive/Documents/2. EPFL/Master/MA2/Image Analysis & Pattern Recognition/iapr24-coin-counter/train/hand_downsampled"
#downsample_images_in_directory(input_directory, output_directory)

Classify background

In [190]:
def classify_background(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    hsv_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    threshold_noisy = ((image[:,:,0] > 60) & (image[:,:,0] < 130))
    threshold_hand = (hsv_image[:,:,0] > 125)

    if (np.sum(threshold_noisy)) > 1000000:
        background = "noisy"
       
    elif np.sum(threshold_hand) > 500000:
        background = "hand"
    
    else:
        background = "neutral"
        
    print(background)
    return background

Segmentation functions

In [191]:
def detect_and_display_circles_hand(img, display=False):
    # Load the image
    """img = cv2.imread(image_path, cv2.IMREAD_COLOR)
    if img is None:
        print("Error: Image not found at", image_path)
        return None"""

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian Blur to the image
    blurred = cv2.GaussianBlur(gray, (9, 9), 2)

    edges = cv2.Canny(blurred, 15, 60, apertureSize=3)
    kernel = np.ones((3, 3), np.uint8)
    edges = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)

    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=90, minLineLength=1, maxLineGap=80)
    mask = np.ones_like(edges) * 255
    if lines is not None:
        for line in lines:
            for x1, y1, x2, y2 in line:
                cv2.line(mask, (x1, y1), (x2, y2), 0, 5)

    edges = cv2.bitwise_and(edges, mask)
    mask_watch = np.ones_like(edges) * 255
    # Mask for the watch
    cv2.rectangle(mask_watch, (0, 850), (1500, 1000), 0, -1)
    edges = cv2.bitwise_and(edges, mask_watch)

    # Apply Hough Circle Transform on the masked image
    circles = cv2.HoughCircles(edges, cv2.HOUGH_GRADIENT, dp=1, minDist=100,
                               param1=150, param2=22, minRadius=50, maxRadius=100)

    if display:
        plt.figure(figsize=(8, 8))
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        if circles is not None:
            circles = np.uint16(np.around(circles))[0, :]
            for (x, y, r) in circles:
                plt.gca().add_patch(plt.Circle((x, y), r, color='red', fill=False, linewidth=2))
        plt.title('Detected Circles in ')
        plt.axis('off')
        plt.show()

    if circles is not None:
        return [(x, y, r) for (x, y, r) in np.uint16(np.around(circles))[0, :]]
    else:
        return []


def detect_and_display_circles_neutral(img, display=False):
    # Load the image
    # img = cv2.imread(image_path, cv2.IMREAD_COLOR)
    # if img is None:
    #    print("Error: Image not found at", image_path)
    #    return None

    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian Blur
    blur = cv2.GaussianBlur(gray, (9, 9), 2)

    # Apply adaptive thresholding and morphological operations
    img_th = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY_INV, 11, 2)
    kernel = np.ones((7, 7), np.uint8)
    closing = cv2.morphologyEx(img_th, cv2.MORPH_CLOSE, kernel)
    closing = cv2.dilate(closing, kernel, iterations=3)

    # Hough Circle Transform
    circles = cv2.HoughCircles(closing, cv2.HOUGH_GRADIENT, dp=1, minDist=100,
                               param1=50, param2=10, minRadius=50, maxRadius=120)

    if display:
        plt.figure(figsize=(8, 8))
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        if circles is not None:
            circles = np.uint16(np.around(circles))[0, :]
            for (x, y, r) in circles:
                plt.gca().add_patch(plt.Circle((x, y), r, color='red', fill=False, linewidth=2))
        plt.title('Detected Circles')
        plt.axis('off')
        plt.show()

    if circles is not None:
        return [(x, y, r) for (x, y, r) in np.uint16(np.around(circles))[0, :]]
    else:
        return []

def detect_and_display_circles_noisy(img, display=False):
    # img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    data_h, data_s, data_v = cv2.split(hsv)
    img_th = np.zeros(data_s.shape, dtype=np.uint8)
    img_th[(data_s > 0.4 * 255) & (data_h < 0.12 * 180)] = 255
    # kernel = np.ones((7, 7), np.uint8)  # Define the kernel size
    # img_th = cv2.morphologyEx(img_th, cv2.MORPH_CLOSE, kernel)
    # plt.imshow(img_th)

    # Apply GaussianBlur to reduce image noise and detail
    blur = cv2.GaussianBlur(img_th, (9, 9), 2)

    # Detect circles in the image using HoughCircles
    """"
    mindist = distance entre 2 cercles :/ => tricky
    param 1 = higher threshold of canny detector => the higher the less edges it detects, bien pour ne pas detecter les unnecessary edges
    param 2 = the smaller it is the more circles it dtects, including false circles

    """
    circles = cv2.HoughCircles(blur, cv2.HOUGH_GRADIENT, dp=1.2, minDist=50, param1=50, param2=40, minRadius=30,
                               maxRadius=100)

    # Convert the (x, y) coordinates and radius of the circles to integers
    if circles is not None:
        circles = np.uint16(np.around(circles))[0, :]

    # Plot the image with circles
    if display:
        plt.figure(figsize=(8, 8))
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        if circles is not None:
        #    circles = np.uint16(np.around(circles))[0, :]
            for (x, y, r) in circles:
                plt.gca().add_patch(plt.Circle((x, y), r, color='red', fill=False, linewidth=2))
        plt.title('Detected Circles')
        plt.axis('off')
        plt.show()

    if circles is not None:
        return [(x, y, r) for (x, y, r) in circles]
    else:
        return []


def crop_coins(image, circles):
    cropped_images = []
    for (x, y, r) in circles:
        x, y, r = int(x), int(y), int(r)
        cropped_img = image[y-r:y+r, x-r:x+r]
        cropped_images.append(cropped_img)
    return cropped_images


def detect_circles_classification(img, display=False):
    # Load the image
    # img = cv2.imread(image_path, cv2.IMREAD_COLOR)
    # if img is None:
    #    print("Error: Image not found at", image_path)
    #    return None

    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian Blur
    blur = cv2.GaussianBlur(gray, (9, 9), 2)

    # Apply adaptive thresholding and morphological operations
    #img_th = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
    #                               cv2.THRESH_BINARY_INV, 11, 2)
    #kernel = np.ones((7, 7), np.uint8)
    #closing = cv2.morphologyEx(img_th, cv2.MORPH_CLOSE, kernel)
    #closing = cv2.dilate(closing, kernel, iterations=3)

    # Hough Circle Transform
    #circles = cv2.HoughCircles(closing, cv2.HOUGH_GRADIENT, dp=1, minDist=100,
    #                           param1=60, param2=10, minRadius=50, maxRadius=120)
    circles = cv2.HoughCircles(blur, cv2.HOUGH_GRADIENT, dp=1, minDist=100,
                               param1=60, param2=10, minRadius=50, maxRadius=120)

    if display:
        plt.figure(figsize=(8, 8))
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        if circles is not None:
            circles = np.uint16(np.around(circles))[0, :]
            for (x, y, r) in circles:
                plt.gca().add_patch(plt.Circle((x, y), r, color='red', fill=False, linewidth=2))
        plt.title('Detected Circles in')
        plt.axis('off')
        plt.show()

    if circles is not None:
        circles = np.uint16(np.around(circles))
        if circles.shape[1] > 1:  # Check if there are multiple circles
            (x, y, r) = circles[0, 0]  # Use the first detected circle
        else:
            (x, y, r) = circles[0, 0]

        return (x, y, r)
    else:
        return (0, 0, 0)


def detect_and_crop_coins(image_type, img=None, img_path=None, display_cropped=False,):
    # Load the image
    if img_path is not None:
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    """if img is None:
        print("Error: Image not found at", image_path)
        return None"""

    # Call the appropriate detection function
    if image_type == 'neutral':
        circles = detect_and_display_circles_neutral(img, display=False)
    elif image_type == 'hand':
        circles = detect_and_display_circles_hand(img, display=False)
    elif image_type == 'noisy':
        circles = detect_and_display_circles_noisy(img, display=False)
    else:
        print("Error: Unknown image type")
        return None

    if not circles:
        print("No circles detected")
        return []

    # Crop the detected coins
    cropped_images = crop_coins(img, circles)

    # Display cropped images if required
    if display_cropped:
        for i, cropped_img in enumerate(cropped_images):
            plt.figure(figsize=(4, 4))
            plt.imshow(cv2.cvtColor(cropped_img, cv2.COLOR_BGR2RGB))
            plt.title(f'Cropped Coin {i + 1}')
            plt.axis('off')
            plt.show()

    return cropped_images, circles

CHF classification functions

In [192]:
def classify_area(area):
    # 5
    if area > 25000:
        return 0
    # 2
    elif area > 17000:
        return 1
    # 1
    elif area > 12000:
        return 2
    # 0.2
    elif area > 10800:
        return 4
    # 0.1
    elif area > 10000:
        return 5
    # 0.5
    elif area > 9000:
        return 3
    # 0.05
    else:
        return 6

In [193]:
def circle_area(circle):
    (x, y, r) = circle
    area = np.pi * r ** 2

    return area

In [194]:
def compute_f1(outputs, labels):
    preds = np.array(outputs)
    labels = np.array(labels)
    score = 0
    for i, true_label in enumerate(labels):
        TP = np.minimum(true_label, preds[i]).sum()
        FPN = np.abs(true_label-preds[i]).sum()
        if 2*TP + FPN != 0:
            score += 2*TP/(2*TP + FPN)

    return score/len(labels)

# 2. Process folder

In [195]:
### TO ADAPT ###
neutral_downsampled = "C:/Users/rapha/OneDrive/Documents/2. EPFL/Master/MA2/Image Analysis & Pattern Recognition/iapr24-coin-counter/train/neutral_downsampled"
noisy_downsampled = "C:/Users/rapha/OneDrive/Documents/2. EPFL/Master/MA2/Image Analysis & Pattern Recognition/iapr24-coin-counter/train/noisy_downsampled"
hand_downsampled = "C:/Users/rapha/OneDrive/Documents/2. EPFL/Master/MA2/Image Analysis & Pattern Recognition/iapr24-coin-counter/train/hand_downsampled"

neutral_folder = "C:/Users/rapha/OneDrive/Documents/2. EPFL/Master/MA2/Image Analysis & Pattern Recognition/iapr24-coin-counter/train/neutral"
noisy_folder = "C:/Users/rapha/OneDrive/Documents/2. EPFL/Master/MA2/Image Analysis & Pattern Recognition/iapr24-coin-counter/train/noisy"
hand_folder = "C:/Users/rapha/OneDrive/Documents/2. EPFL/Master/MA2/Image Analysis & Pattern Recognition/iapr24-coin-counter/train/hand"

test_folder = "C:/Users/rapha/OneDrive/Documents/2. EPFL/Master/MA2/Image Analysis & Pattern Recognition/iapr24-coin-counter/test"

model_path1 = "C:/Users/rapha/OneDrive/Documents/2. EPFL/Master/MA2/Image Analysis & Pattern Recognition/iapr_group_55/project/classifier_wholetrainset_class3_b6_lr2e3_step7_epoch11_split0.75"
model_path2 = "C:/Users/rapha/OneDrive/Documents/2. EPFL/Master/MA2/Image Analysis & Pattern Recognition/iapr_group_55/project/resnet_b6_lr1e2_step7_epoch8_split0.75"

csv_file = "C:/Users/rapha/OneDrive/Documents/2. EPFL/Master/MA2/Image Analysis & Pattern Recognition/iapr24-coin-counter/train_labels.csv"

Choose folder to process

In [196]:
### TO ADAPT ###
folder1 = neutral_downsampled # ! Changer le background dans load_images_and_backgrounds
folder2 = neutral_folder

Model CHF/EUR/OOD (alexnet)

In [197]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms

model1 = torchvision.models.alexnet(weights='IMAGENET1K_V1')
num_ftrs = model1.classifier[-1].in_features
model1.classifier[-1] = nn.Linear(num_ftrs, 3)
model1.load_state_dict(torch.load(model_path1))

<All keys matched successfully>

Model EUR subclasses (resnet)

In [198]:
import random

torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

model2 = torchvision.models.resnet18(weights='IMAGENET1K_V1')

num_ftrs = model2.fc.in_features
model2.fc = nn.Linear(num_ftrs, 8)

model2.load_state_dict(torch.load(model_path2))

<All keys matched successfully>

In [199]:
preprocess = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),  # Resize to match model's input size
    transforms.ToTensor(),  # Convert PIL image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # Normalize pixel values, based on imagenet
])

Background

In [200]:
def load_images_and_backgrounds(folder, downsampled=True):
    images = []
    ids = []
    backgrounds = []

    for root, dirs, files in os.walk(folder):
        for file in files:
            if file.endswith('.JPG'):
                print(f"Processing file {file}")
                # Construct path to the image file
                file_path = os.path.join(root, file)
                im = cv2.imread(file_path, cv2.IMREAD_COLOR)
                img_id = file[:-4]

                # Classify background
                if not downsampled:
                    # Classify and downsample
                    background = classify_background(im)
                    size=(1500, 1000)
                    im = cv2.resize(im, size)
                else:
                    # Manually assign background
                    background = 'neutral'

                images.append(im)
                ids.append(img_id)
                backgrounds.append(background)

    return images, ids, backgrounds

images1, ids1, backgrounds1 = load_images_and_backgrounds(folder1, downsampled=True)
images2, ids2, backgrounds2 = load_images_and_backgrounds(folder2, downsampled=False)

Processing file L1010277.JPG
Processing file L1010279.JPG
Processing file L1010281.JPG
Processing file L1010283.JPG
Processing file L1010287.JPG
Processing file L1010288.JPG
Processing file L1010294.JPG
Processing file L1010297.JPG
Processing file L1010298.JPG
Processing file L1010300.JPG
Processing file L1010308.JPG
Processing file L1010310.JPG
Processing file L1010311.JPG
Processing file L1010318.JPG
Processing file L1010321.JPG
Processing file L1010323.JPG
Processing file L1010405.JPG
Processing file L1010406.JPG
Processing file L1010408.JPG
Processing file L1010410.JPG
Processing file L1010413.JPG
Processing file L1010418.JPG
Processing file L1010419.JPG
Processing file L1010421.JPG
Processing file L1010422.JPG
Processing file L1010424.JPG
Processing file L1010434.JPG
Processing file L1010436.JPG
Processing file L1010440.JPG
Processing file L1010445.JPG
Processing file L1010446.JPG
Processing file L1010450.JPG
Processing file L1010454.JPG
Processing file L1010277.JPG
neutral
Proces

Predictions

In [201]:
def predict(images, ids, backgrounds):
    predictions = {}
    subclasses = {}

    model1.eval()
    model2.eval()

    for i, (background, img) in enumerate(zip(backgrounds,images)):
        print(background)
        cropped_images, circles = detect_and_crop_coins(background, np.array(img))

        pred = np.zeros(3)
        subclass = np.zeros(16)
        outputs = []

        if len(cropped_images) != 0:
            for coin, circle in zip(cropped_images, circles):
                new_circle = detect_circles_classification(coin)
                coin = preprocess(coin)
                coin = coin.unsqueeze(0)

                with torch.no_grad():
                    outputs = model1(coin)
                output = torch.argmax(outputs) # index 0 -> CHF, index 1 -> EUR, index 2 -> OOD
                print(output)
                
                if output == 0:
                    area = circle_area(new_circle)
                    label = classify_area(area)
                
                if output == 1:
                    with torch.no_grad():
                        outputs2 = model2(coin) # index 0 -> 0.01EUR, ..., index 7 -> 2EUR
                    inverted_label = 7 - torch.argmax(outputs2).item() # index 0 -> 2EUR, ..., index 7 -> 0.01EUR
                    label = 7 + inverted_label # index 7 -> 2EUR, ..., index 14 -> 0.01EUR

                if output == 2:
                    label = 15
                
                subclass[label] += 1 
                subclasses[ids[i]] = subclass
                pred[output] += 1 # add the coin to the image prediction

        else:
            print(f'no coins detected in image {i}')

        predictions[ids[i]] = pred
        
    return predictions, subclasses

predictions1, subclasses1 = predict(images1, ids1, backgrounds1)
predictions2, subclasses2 = predict(images2, ids2, backgrounds2)

neutral
tensor(2)
tensor(2)
tensor(0)
tensor(0)
tensor(0)
neutral
tensor(2)
tensor(2)
tensor(0)
tensor(0)
neutral
tensor(0)
tensor(0)
tensor(2)
tensor(2)
tensor(2)
tensor(0)
tensor(2)
tensor(0)
neutral
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
neutral
tensor(2)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(2)
neutral
tensor(0)
tensor(2)
tensor(0)
tensor(0)
tensor(0)
neutral
tensor(2)
tensor(2)
tensor(0)
tensor(0)
tensor(2)
neutral
tensor(2)
tensor(0)
tensor(0)
tensor(0)
neutral
tensor(2)
tensor(2)
tensor(0)
tensor(0)
tensor(2)
neutral
tensor(2)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
neutral
tensor(2)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
neutral
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
neutral
tensor(2)
tensor(2)
tensor(0)
tensor(2)
tensor(0)
tensor(0)
neutral
tensor(2)
tensor(2)
tensor(0)
tensor(2)
tensor(2)
tensor(0)
tensor(0)
tensor(0)
tensor(2)
tensor(0)
tensor(0)
neutral


In [202]:
predictions1

{'L1010277': array([3., 0., 2.]),
 'L1010279': array([2., 0., 2.]),
 'L1010281': array([4., 0., 4.]),
 'L1010283': array([6., 0., 0.]),
 'L1010287': array([6., 0., 2.]),
 'L1010288': array([4., 0., 1.]),
 'L1010294': array([2., 0., 3.]),
 'L1010297': array([3., 0., 1.]),
 'L1010298': array([2., 0., 3.]),
 'L1010300': array([4., 0., 1.]),
 'L1010308': array([8., 0., 1.]),
 'L1010310': array([7., 0., 0.]),
 'L1010311': array([3., 0., 3.]),
 'L1010318': array([6., 0., 5.]),
 'L1010321': array([8., 0., 1.]),
 'L1010323': array([8., 0., 0.]),
 'L1010405': array([2., 1., 1.]),
 'L1010406': array([3., 0., 2.]),
 'L1010408': array([0., 0., 2.]),
 'L1010410': array([1., 0., 2.]),
 'L1010413': array([3., 0., 1.]),
 'L1010418': array([1., 0., 1.]),
 'L1010419': array([2., 0., 1.]),
 'L1010421': array([3., 0., 1.]),
 'L1010422': array([3., 0., 0.]),
 'L1010424': array([5., 0., 0.]),
 'L1010434': array([2., 0., 2.]),
 'L1010436': array([1., 0., 0.]),
 'L1010440': array([6., 0., 2.]),
 'L1010445': a

In [203]:
predictions2

{'L1010277': array([2., 0., 3.]),
 'L1010279': array([0., 0., 4.]),
 'L1010281': array([3., 0., 5.]),
 'L1010283': array([3., 0., 3.]),
 'L1010287': array([5., 0., 3.]),
 'L1010288': array([3., 0., 2.]),
 'L1010294': array([2., 0., 3.]),
 'L1010297': array([0., 0., 4.]),
 'L1010298': array([1., 0., 4.]),
 'L1010300': array([1., 0., 4.]),
 'L1010308': array([8., 0., 1.]),
 'L1010310': array([4., 0., 3.]),
 'L1010311': array([3., 0., 3.]),
 'L1010318': array([5., 0., 6.]),
 'L1010321': array([5., 0., 4.]),
 'L1010323': array([7., 0., 1.]),
 'L1010405': array([2., 1., 1.]),
 'L1010406': array([3., 0., 2.]),
 'L1010408': array([0., 0., 2.]),
 'L1010410': array([1., 0., 2.]),
 'L1010413': array([2., 0., 2.]),
 'L1010418': array([1., 0., 1.]),
 'L1010419': array([2., 0., 1.]),
 'L1010421': array([2., 0., 2.]),
 'L1010422': array([3., 0., 0.]),
 'L1010424': array([2., 0., 3.]),
 'L1010434': array([2., 0., 2.]),
 'L1010436': array([0., 0., 1.]),
 'L1010440': array([6., 0., 2.]),
 'L1010445': a

In [204]:
subclasses = subclasses1

CHF predictions

In [205]:
import pandas as pd
# Load the CSV file into a pandas DataFrame
df = pd.read_csv(csv_file)
#df.drop(df.columns[8:17], axis=1, inplace=True)

selected_rows = df[df['id'].isin(subclasses.keys())]
selected_rows.shape

(33, 17)

In [206]:
# Initialize variables to store total correct predictions and total elements
total_correct = 0
total_elements = 0
all_true_labels = []
all_pred_labels = []

# Iterate over each key in subclasses
for key in subclasses.keys():
    # Extract the row values from the DataFrame
    row_values = df.loc[df['id'] == key].iloc[0, 1:17].tolist()

    # Count the number of element-wise equal elements
    equal_count = sum(1 for x, y in zip(row_values, subclasses[key]) if x == y)

    # Increment total correct and total elements
    total_correct += equal_count
    total_elements += len(subclasses[key])

    print(key)
    print(row_values)
    print(subclasses[key])

    # Collect true labels and predicted labels for F1 score computation
    all_true_labels.append(subclasses[key])
    all_pred_labels.append(row_values)

# Calculate total accuracy
total_accuracy = (total_correct / total_elements) * 100

# Compute F1 score using the custom function
f1 = compute_f1(all_pred_labels, all_true_labels)

print(f"Total accuracy: {total_accuracy:.2f}%")
print(f"F1 score: {f1:.4f}")

L1010277
[1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 2, 0, 0, 0, 0, 0]
[0. 0. 3. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2.]
L1010279
[0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2.]
L1010281
[0, 0, 0, 0, 2, 3, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0]
[0. 0. 2. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 4.]
L1010283
[0, 0, 0, 1, 0, 0, 0, 1, 0, 2, 1, 0, 1, 0, 0, 0]
[0. 1. 4. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
L1010287
[0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 2, 1, 1, 1, 0]
[0. 0. 2. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 2.]
L1010288
[0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0]
[0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
L1010294
[1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]
[0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 3.]
L1010297
[0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0]
[0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
L1010298
[0, 2, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 3.]
L1010300
[0, 1, 0, 0, 0, 0, 